In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from pathlib import Path
import pandas as pd
from tqdm import tqdm
import json
import numpy as np
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
import openslide
import time

In [3]:
## HACK get rid of it
import sys
sys.path.append("/home/bronzi/ProgProjekte/Re-stained_WSIs_Registration/src")

from wsi_registration import TissueDetector, MatcherParameters, WSI_Matcher

In [4]:
slides = {path.stem: path for path in Path(r"/mnt/d/Datasets/ScannerStudy").glob("*/*/*.tiff")}
slides

{'A_CCMCT_183715A_1': PosixPath('/mnt/d/Datasets/ScannerStudy/Aperio/CCMCT/A_CCMCT_183715A_1.tiff'),
 'A_CCMCT_22108_1': PosixPath('/mnt/d/Datasets/ScannerStudy/Aperio/CCMCT/A_CCMCT_22108_1.tiff'),
 'A_CCMCT_29609B_1': PosixPath('/mnt/d/Datasets/ScannerStudy/Aperio/CCMCT/A_CCMCT_29609B_1.tiff'),
 'A_CCMCT_380609B_1': PosixPath('/mnt/d/Datasets/ScannerStudy/Aperio/CCMCT/A_CCMCT_380609B_1.tiff'),
 'A_CCMCT_518711B_1': PosixPath('/mnt/d/Datasets/ScannerStudy/Aperio/CCMCT/A_CCMCT_518711B_1.tiff'),
 'Z_CCMCT_183715A_1': PosixPath('/mnt/d/Datasets/ScannerStudy/AxioScan/CCMCT/Z_CCMCT_183715A_1.tiff'),
 'Z_CCMCT_22108_1': PosixPath('/mnt/d/Datasets/ScannerStudy/AxioScan/CCMCT/Z_CCMCT_22108_1.tiff'),
 'Z_CCMCT_29609B_1': PosixPath('/mnt/d/Datasets/ScannerStudy/AxioScan/CCMCT/Z_CCMCT_29609B_1.tiff'),
 'Z_CCMCT_380609B_1': PosixPath('/mnt/d/Datasets/ScannerStudy/AxioScan/CCMCT/Z_CCMCT_380609B_1.tiff'),
 'Z_CCMCT_518711B_1': PosixPath('/mnt/d/Datasets/ScannerStudy/AxioScan/CCMCT/Z_CCMCT_518711B_1.

In [5]:
def add_help_fields(frame):
    
    frame["image_name_stem"] = [Path(image_name).stem for image_name in frame["image_name"]]
    
    frame["patient_id"] = [name.split("_")[2] for name in frame["image_name"]]

    frame["x1"] = [json.loads(vector.replace("\'","\""))['x1'] for vector in frame["vector"]]
    frame["y1"] = [json.loads(vector.replace("\'","\""))['y1'] for vector in frame["vector"]]

    frame["x2"] = [json.loads(vector.replace("\'","\""))['x2'] for vector in frame["vector"]]
    frame["y2"] = [json.loads(vector.replace("\'","\""))['y2'] for vector in frame["vector"]]

    frame["center_x"] = [x1 + ((x2-x1) / 2) for x1, x2 in zip(frame["x1"], frame["x2"])]
    frame["center_y"] = [y1 + ((y2-y1) / 2) for y1, y2 in zip(frame["y1"], frame["y2"])]
    
    frame["center"] = [[center_x, center_y] for center_x, center_y in zip(frame["center_x"], frame["center_y"])]

    frame["anno_width"] = [x2-x1 for x1, x2 in zip(frame["x1"], frame["x2"])]
    frame["anno_height"]= [y2-y1 for y1, y2 in zip(frame["y1"], frame["y2"])]
    
    return frame

In [6]:
gt_annotations = add_help_fields(pd.read_csv("GT.csv"))

gt_annotations = gt_annotations[gt_annotations["image_type"] == "CCMCT"]

gt_annotations.head()

,scanner,image_id,image_name,image_type,image_width,image_height,id,vector,unique_identifier,annotation_type,...,patient_id,x1,y1,x2,y2,center_x,center_y,center,anno_width,anno_height
0,2.0HT,10410,N2_CCMCT_380609B_1.ndpi,CCMCT,147456,107008,2755406,"{'x1': 20354, 'x2': 20408, 'y1': 16752, 'y2': ...",95183f6e-0a00-4de4-9c97-fedfec655a27,384,...,380609B,20354,16752,20408,16806,20381.0,16779.0,"[20381.0, 16779.0]",54,54
1,2.0HT,10410,N2_CCMCT_380609B_1.ndpi,CCMCT,147456,107008,2755407,"{'x1': 20256, 'x2': 20310, 'y1': 35448, 'y2': ...",cc3def2b-5631-4619-ab08-51f2a69c667d,385,...,380609B,20256,35448,20310,35502,20283.0,35475.0,"[20283.0, 35475.0]",54,54
2,2.0HT,10410,N2_CCMCT_380609B_1.ndpi,CCMCT,147456,107008,2755408,"{'x1': 20445, 'x2': 20499, 'y1': 54139, 'y2': ...",2d3de917-e55a-470f-8af2-10eb49b69687,386,...,380609B,20445,54139,20499,54193,20472.0,54166.0,"[20472.0, 54166.0]",54,54
3,2.0HT,10410,N2_CCMCT_380609B_1.ndpi,CCMCT,147456,107008,2755409,"{'x1': 26495, 'x2': 26549, 'y1': 71854, 'y2': ...",611f9d35-6cdb-42ef-bc38-2f44c9aaa257,387,...,380609B,26495,71854,26549,71908,26522.0,71881.0,"[26522.0, 71881.0]",54,54
4,2.0HT,10410,N2_CCMCT_380609B_1.ndpi,CCMCT,147456,107008,2755410,"{'x1': 35443, 'x2': 35497, 'y1': 90091, 'y2': ...",bf3ef0d3-5153-4693-96ba-96047cb3ae24,388,...,380609B,35443,90091,35497,90145,35470.0,90118.0,"[35470.0, 90118.0]",54,54


In [12]:
results = []


tissue_detector = TissueDetector("LAB_Threshold", threshold=90)
matcher_parameters = MatcherParameters() 

for patient_id in tqdm(gt_annotations["patient_id"].unique()):
    
    patient_annos = gt_annotations[gt_annotations["patient_id"] == patient_id]
    
    source_annos = patient_annos[patient_annos["scanner"] == "Aperio"]        
    
    intersections = list(set(source_annos["type_name"]).intersection(patient_annos["type_name"]))
    
    source_annos = source_annos[source_annos["type_name"].isin(intersections)]
    
    source_anno = source_annos.iloc[0]
    
    for scanner in ['2.0HT', 'Axio', 'S210']:
        
        target_annos = patient_annos[patient_annos["scanner"] == scanner]
        target_annos = target_annos[target_annos["type_name"].isin(intersections)]
        
        target_anno = target_annos.iloc[0]      
        
        target_scanner = target_anno.scanner 
        image_id = target_anno.image_id 
        image_name = target_anno.image_name 
        image_width, image_height = target_anno.image_width, target_anno.image_height
        
        image_type = "CCMCT"
        
        try:
        
            start = time.time()
            
            source_slide = slides[Path(source_anno.image_name).stem]
            target_slide = slides[Path(target_anno.image_name).stem]

            matcher = WSI_Matcher(tissue_detector, matcher_parameters)
            offset = matcher.match(str(source_slide), str(target_slide))

            print(f"S: {source_slide.stem}  T: {target_slide.stem} Results: {offset}")

            end = time.time()

            run_time = end - start

            for id, source_anno in source_annos.iterrows():          

                target_scanner = target_anno.scanner
                image_id = target_anno.image_id
                image_name = target_anno.image_name
                image_width, image_height = target_anno.image_width, target_anno.image_height

                new_x1, new_y1 = source_anno.x1 + offset[0], source_anno.y1 + offset[1] 
                new_x2, new_y2 = source_anno.x2 + offset[0], source_anno.y2 + offset[1] 

                vector = {'x1': new_x1, 'x2': new_x2, 'y1': new_y1, 'y2': new_y2}

                row = [target_scanner, image_id, image_name, image_type, image_width, image_height,
                           vector, source_anno.unique_identifier, source_anno.annotation_type, 
                           source_anno.type_name, f"ReStain", run_time, 1024]

                results.append(row)
        except:
            print(f"S: {source_anno.image_name}  T: {target_anno.image_name} Results: {None, None}")

  0%|          | 0/5 [00:00<?, ?it/s]

S: A_CCMCT_380609B_1  T: N2_CCMCT_380609B_1 Results: (8457.085414919595, 2672.2650280806915)
S: A_CCMCT_380609B_1  T: Z_CCMCT_380609B_1 Results: (-3695.36875847768, 5626.654444562913)


 20%|██        | 1/5 [00:53<03:32, 53.17s/it]

S: A_CCMCT_380609B_1  T: N1_CCMCT_380609B_1 Results: (3005.972021847778, 4494.061910611861)
S: A_CCMCT_518711B_1  T: N2_CCMCT_518711B_1 Results: (1845.1196592752192, -776.8965310272365)
S: A_CCMCT_518711B_1  T: Z_CCMCT_518711B_1 Results: (-3386.2510802936886, -1117.4631693405183)


 40%|████      | 2/5 [01:56<02:57, 59.11s/it]

...

 80%|████████  | 4/5 [04:00<01:01, 61.52s/it]

S: A_CCMCT_29609B_1  T: N1_CCMCT_29609B_1 Results: (42292.12438354025, 2798.4758633730894)
S: A_CCMCT_22108_1  T: N2_CCMCT_22108_1 Results: (2015.4269899611938, -1030.2257271213305)
S: A_CCMCT_22108_1  T: Z_CCMCT_22108_1 Results: (-3198.1614109404254, -2293.917002845636)


100%|██████████| 5/5 [05:18<00:00, 63.69s/it]

S: A_CCMCT_22108_1  T: N1_CCMCT_22108_1 Results: (-1213.602611838286, 5954.497493304608)


ValueError: 11 columns passed, passed data had 13 columns

In [13]:
results = pd.DataFrame(results, columns=["scanner", "image_id", "image_name", "image_type", "image_width", 
                                         "image_height", "vector", "unique_identifier", "annotation_type", "type_name", "method", 
                                        "runtime", "thumbnail_size"])

results.to_csv(f"ReStain.csv", index=False)

results.head()

,scanner,image_id,image_name,image_type,image_width,image_height,vector,unique_identifier,annotation_type,type_name,method,runtime,thumbnail_size
0,2.0HT,10410,N2_CCMCT_380609B_1.ndpi,CCMCT,147456,107008,"{'x1': 23905.085414919595, 'x2': 23955.0854149...",95183f6e-0a00-4de4-9c97-fedfec655a27,384,L0,ReStain,15.335656,1024
1,2.0HT,10410,N2_CCMCT_380609B_1.ndpi,CCMCT,147456,107008,"{'x1': 23765.085414919595, 'x2': 23815.0854149...",cc3def2b-5631-4619-ab08-51f2a69c667d,385,L1,ReStain,15.335656,1024
2,2.0HT,10410,N2_CCMCT_380609B_1.ndpi,CCMCT,147456,107008,"{'x1': 23885.085414919595, 'x2': 23935.0854149...",2d3de917-e55a-470f-8af2-10eb49b69687,386,L2,ReStain,15.335656,1024
3,2.0HT,10410,N2_CCMCT_380609B_1.ndpi,CCMCT,147456,107008,"{'x1': 29249.085414919595, 'x2': 29299.0854149...",611f9d35-6cdb-42ef-bc38-2f44c9aaa257,387,L3,ReStain,15.335656,1024
4,2.0HT,10410,N2_CCMCT_380609B_1.ndpi,CCMCT,147456,107008,"{'x1': 37193.085414919595, 'x2': 37243.0854149...",bf3ef0d3-5153-4693-96ba-96047cb3ae24,388,L4,ReStain,15.335656,1024
